In [9]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="dataset/snli_1.0_train.jsonl")

In [10]:
dataset = dataset["train"]

In [11]:
dataset = dataset.filter(lambda example: example["gold_label"] in ["contradiction", "neutral", "entailment"])

In [12]:
dataset

Dataset({
    features: ['annotator_labels', 'captionID', 'gold_label', 'pairID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse'],
    num_rows: 549367
})

In [13]:
import numpy as np

neigh_idx = np.load("retriever/neigh_idx.npy")

In [14]:
neigh_idx

array([[100430, 100429, 513844],
       [100430, 258532, 120379],
       [378292, 149725,   5931],
       ...,
       [487228, 202967, 113237],
       [487228,  25246, 221644],
       [482706, 482872, 196670]])

In [15]:
sentence1 = np.array(dataset["sentence1"])
sentence2 = np.array(dataset["sentence2"])
sentence3 = np.array(dataset["gold_label"])

In [16]:

# 假设 sentence1, sentence2, sentence3 都是长度为 9824 的列表
# neigh_idx 是形状为 (9824, 3) 的 ndarray

# 定义一个方法来构建每一行的字典
def create_dicts_for_row(idx1, idx2, idx3, sentence1, sentence2, sentence3):
    return [
        {"premise": sentence1[idx1], "hypothesis": sentence2[idx1], "relationship": sentence3[idx1].title()},
        {"premise": sentence1[idx2], "hypothesis": sentence2[idx2], "relationship": sentence3[idx2].title()},
        {"premise": sentence1[idx3], "hypothesis": sentence2[idx3], "relationship": sentence3[idx3].title()},
    ]

# 使用列表推导式和 itertools.starmap 优化生成过程
result = [
    create_dicts_for_row(idx1, idx2, idx3, sentence1, sentence2, sentence3)
    for idx1, idx2, idx3 in neigh_idx
]
result

[[{'premise': 'This local musical ensemble stands for their final performance of the day. in a gazebo in a park.',
   'hypothesis': 'The musicians stand at their mics on the stage at the bar.',
   'relationship': 'Contradiction'},
  {'premise': 'This local musical ensemble stands for their final performance of the day. in a gazebo in a park.',
   'hypothesis': 'The ensemble includes a flute, violin, and oboe.',
   'relationship': 'Neutral'},
  {'premise': 'This picture is a strange combination of old times and present day.',
   'hypothesis': 'The picture is from the late 16th century.',
   'relationship': 'Contradiction'}],
 [{'premise': 'This local musical ensemble stands for their final performance of the day. in a gazebo in a park.',
   'hypothesis': 'The musicians stand at their mics on the stage at the bar.',
   'relationship': 'Contradiction'},
  {'premise': 'This is a boy that is a guest at Jewish outing.',
   'hypothesis': 'The boy is attending a First Holy Communion at the Cat